### 
1. Link to get the token: https://learn.microsoft.com/en-us/rest/api/power-bi/embed-token/generate-token#code-try-0
2. Tenant_id is the group id of the report

In [ ]:
import requests
import pandas as pd

In [ ]:
# Replace with your tenant ID, client ID, and client secret
tenant_id = ''      # Input tenant_id
# scope = 'https://analysis.windows.net/powerbi/api/.default'

access_token = ''
print(f'Bearer Token: {access_token}')

In [ ]:
# Get the list of reports
def get_report_list(tenant_id, access_token):
    reports_url = f'https://api.powerbi.com/v1.0/myorg/groups/{tenant_id}/reports'
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    reports_list = requests.get(reports_url, headers=headers, verify=False)
     # Check if the request was successful
    if reports_list.status_code == 200:
        return reports_list.json().get('value')
    else:
        print(f"Failed to retrieve reports. Status Code: {reports_list.status_code}")
        print(f"Response: {reports_list.text}")
        return None

In [ ]:
# Get the list of the report
reports = get_report_list(tenant_id, access_token)

# Loop through each report and get the users
report_data = []
for report in reports:
    report_id = report['id']
    report_name = report['name']
    dataset_id = report['datasetId']
    report_data.append({'Report ID': report_id, 'Report Name': report_name, 'Dataset ID': dataset_id})  #, 'Group ID': group_id})
    # print(f'Report ID: {report_id}, Report Name: {report_name}')


# Convert to DataFrame for better readability (optional)
report_df = pd.DataFrame(report_data)
report_df.head()

In [ ]:
filtered_df = report_df.loc[report_df['Dataset ID'] == 'c7898c32-e697-4256-9d6e-b222126f3335']
filtered_df

In [ ]:
# Function to get the all the dataset id
def get_report_users(tenant_id, access_token):
    users_url = f'https://api.powerbi.com/v1.0/myorg/groups/{tenant_id}/datasets/{dataset_id}/users'     #'https://api.powerbi.com/v1.0/myorg/admin/reports/{report_id}/users'
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    response = requests.get(users_url, headers=headers, verify=False)

    if response.status_code == 200:
        return response.json().get('value')
    else:
        return None

In [ ]:
# passing dataset_id column into a list

dataset_list = report_df['Dataset ID'].to_list()
print(dataset_list)

In [ ]:
# Loop through each dataset ID and get the users
all_users_data = []
for dataset_id in dataset_list:
    users = get_report_users(tenant_id, access_token)
    if users is not None:
        for user in users:
            all_users_data.append({
                'Dataset ID': dataset_id,
                'User': user['identifier'],
                'Access Right': user['datasetUserAccessRight'],
                'Principal Type': user['principalType']
            })

# Convert to DataFrame for better readability (optional)
dataset_df = pd.DataFrame(all_users_data)
print(dataset_df.head())

In [ ]:
dataset_df.head()

In [ ]:
combined_df = pd.merge(report_df, dataset_df, on='Dataset ID', how='inner')
combined_df.head()

In [ ]:
combined_df.to_csv('output.csv', index=False)